In [19]:
import sys
import json
import math
import argparse
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline
import random

In [20]:
file_ftm_nobody = "log.ftm.nobody.csv"

file_ftm_1p = "log.ftm.1person.csv"

file_ftm_2p = "log.ftm.2person.csv"


In [21]:
def read_file(filename):
    lines = []
    with open(filename) as f:
        while True:
            line = f.readline()
            components = line.split(",")
            if line != '':
                lines.append(components)
            if not line:
                break;
    return lines

In [22]:

def get_data_pos(char_string, char_compare):
    for i in range(len(char_string)):
        if char_string[i] == char_compare:
            return i

In [23]:
def process_ftm(components):
#     print(components)
    channel = components[1]
    mac = components[2]
    rtt_time = components[3]

    j = get_data_pos(components[4], '[')

    rtt_dist = components[4][1:j-2]
    # collect rtt data:
    rtt_data = []
    rtt_data.append(components[4][j+1:-1])

    for i in range(5, len(components)-1):
        rtt_data.append(components[i])

    m = get_data_pos(components[-1], ']')
    rtt_data.append(components[-1][1:m])
    
    return channel, mac, rtt_time, rtt_dist, rtt_data


In [24]:
def generate_dataframe_w_truth(lines, divide_index, truth):
    parsed_data = []
    for i in range(len(lines)):
        channel, mac, rtt_time, rtt_dist, rtt_data = process_ftm(lines[i])
        parsed_data.append([channel, mac, rtt_time, rtt_dist, rtt_data])

    df_ftm = pd.DataFrame(parsed_data, columns=['channel', 'mac', 'rtt_time', 'rtt_dist', 'rtt_data'])
    
    ftm_w_truth = []
    for i in range(len(lines)//divide_index):
        temp_lst = []
        for j in range(divide_index):
            temp_lst.append(float(df_ftm.iloc[10*i+j]['rtt_dist']))
        ftm_w_truth.append([temp_lst, truth])
#     ftm_w_truth = pd.DataFrame(ftm_w_truth, columns = ['rtt_dist', 'ground truth'])
#     print(ftm_w_truth)
    return ftm_w_truth

In [25]:
#read file and generate 
lines_nobody = read_file(file_ftm_nobody)
ftm_nobody_w_truth = generate_dataframe_w_truth(lines_nobody, 10, 0)

lines_1p = read_file(file_ftm_1p)
ftm_1p_w_truth = generate_dataframe_w_truth(lines_1p, 10, 1)

lines_2p = read_file(file_ftm_2p)
ftm_2p_w_truth = generate_dataframe_w_truth(lines_2p, 10, 2)

In [26]:
# combine and shuffle all data
all_ftm_data = []
for i in range(len(ftm_nobody_w_truth)):
    all_ftm_data.append(ftm_nobody_w_truth[i])
    all_ftm_data.append(ftm_1p_w_truth[i])
    all_ftm_data.append(ftm_2p_w_truth[i])
    
random.shuffle(all_ftm_data)
# 到这儿所有的data处理应该都完成了，每个list中每个元素的格式是：【【十个数，每channel的ftm reading】，0/1/2取决于ground truth】

In [27]:
with open('ftm_processed_data.csv', 'w') as filehandle:
    filehandle.writelines("%s\n" % place for place in all_ftm_data)
filehandle.close()